# Classificação de Sinais Vitais - Resgate de Vítimas de Catástrofes

**Trabalho 3 - Curso de Especialização em Ciência de Dados 2025**

Aluno: Vinícius de Souza Cebalhos

## Objetivo

Classificar vítimas de acidentes em 4 classes de gravidade baseado em sinais vitais:
- **Classe 1**: Crítico
- **Classe 2**: Instável  
- **Classe 3**: Potencialmente Estável
- **Classe 4**: Estável

## Tarefa

Comparar dois modelos de classificação:
1. **Random Forest** (Florestas Aleatórias)
2. **MLP** (Multi-Layer Perceptron - Rede Neural Artificial)

Usando métricas: Precisão, Recall, F1-Score, Acurácia e Matriz de Confusão


In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            confusion_matrix, classification_report)
import warnings

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
np.random.seed(42)

print("Bibliotecas importadas com sucesso!")


## 1. Carregamento e Auditoria dos Dados


In [ ]:
# Carregamento do arquivo de treino
print("=" * 80)
print("CARREGAMENTO DOS DADOS DE TREINO")
print("=" * 80)

file_path = 'treino_sinais_vitais_com_label_1500.txt'

try:
    # Tentar carregar o arquivo
    # Formato: i,si1,si2,si3,si4,si5,gi,yi (separado por vírgulas)
    df_train = pd.read_csv(file_path, sep=',', header=None, 
                          names=['i', 'si1', 'si2', 'si3', 'si4', 'si5', 'gi', 'yi'])
    
    print(f"\n✅ Arquivo carregado com sucesso!")
    print(f"Shape: {df_train.shape}")
    print(f"Linhas: {df_train.shape[0]}, Colunas: {df_train.shape[1]}\n")
    
    print("Primeiras linhas do dataset:")
    print(df_train.head(10))
    
    print("\nÚltimas linhas do dataset:")
    print(df_train.tail(10))
    
except FileNotFoundError:
    print(f"⚠️ Arquivo '{file_path}' não encontrado!")
    print("Por favor, certifique-se de que o arquivo está no diretório correto.")
    print("\nEstrutura esperada do arquivo:")
    print("i si1 si2 si3 si4 si5 gi yi")
    print("1 valor1 valor2 valor3 valor4 valor5 valor6 classe")
    df_train = None
except Exception as e:
    print(f"⚠️ Erro ao carregar arquivo: {e}")
    df_train = None


In [ ]:
# Auditoria inicial dos dados
if df_train is not None:
    print("=" * 80)
    print("AUDITORIA INICIAL DOS DADOS")
    print("=" * 80)
    
    print("\n1. INFORMAÇÕES GERAIS:")
    print(df_train.info())
    
    print("\n2. ESTATÍSTICAS DESCRITIVAS:")
    print(df_train.describe())
    
    print("\n3. VERIFICAÇÃO DE VALORES AUSENTES:")
    missing = df_train.isnull().sum()
    if missing.sum() > 0:
        print("⚠️ Valores ausentes encontrados:")
        print(missing[missing > 0])
    else:
        print("✅ Nenhum valor ausente encontrado")
    
    print("\n4. VERIFICAÇÃO DE DUPLICATAS:")
    duplicatas = df_train.duplicated().sum()
    if duplicatas > 0:
        print(f"⚠️ {duplicatas} linhas duplicadas encontradas")
    else:
        print("✅ Nenhuma linha duplicada")
    
    print("\n5. DISTRIBUIÇÃO DAS CLASSES (yi):")
    print(df_train['yi'].value_counts().sort_index())
    print(f"\nDistribuição percentual:")
    print((df_train['yi'].value_counts(normalize=True) * 100).sort_index())
    
    print("\n6. VERIFICAÇÃO DE FAIXAS ESPERADAS:")
    print("\nsi1 (pressão sistólica): esperado [5, 22]")
    print(f"  Observado: [{df_train['si1'].min():.2f}, {df_train['si1'].max():.2f}]")
    
    print("\nsi2 (pressão diastólica): esperado [0, 15]")
    print(f"  Observado: [{df_train['si2'].min():.2f}, {df_train['si2'].max():.2f}]")
    
    print("\nsi3 (qPA - qualidade pressão): esperado [-10, 10]")
    print(f"  Observado: [{df_train['si3'].min():.2f}, {df_train['si3'].max():.2f}]")
    
    print("\nsi4 (pulso): esperado [0, 200] bpm")
    print(f"  Observado: [{df_train['si4'].min():.2f}, {df_train['si4'].max():.2f}]")
    
    print("\nsi5 (respiração): esperado [0, 22] FpM")
    print(f"  Observado: [{df_train['si5'].min():.2f}, {df_train['si5'].max():.2f}]")
    
    print("\n7. VERIFICAÇÃO DE VALORES NEGATIVOS (onde não deveria haver):")
    if (df_train['si4'] < 0).sum() > 0:
        print(f"⚠️ {((df_train['si4'] < 0).sum())} valores negativos em si4 (pulso)")
    else:
        print("✅ si4 (pulso): nenhum valor negativo")
    
    if (df_train['si5'] < 0).sum() > 0:
        print(f"⚠️ {((df_train['si5'] < 0).sum())} valores negativos em si5 (respiração)")
    else:
        print("✅ si5 (respiração): nenhum valor negativo")


In [ ]:
# Análise Exploratória
if df_train is not None:
    print("=" * 80)
    print("ANÁLISE EXPLORATÓRIA DOS DADOS")
    print("=" * 80)
    
    # Features que DEVEM ser usadas (conforme enunciado)
    features_usar = ['si3', 'si4', 'si5', 'gi']  # si1 e si2 NÃO devem ser usadas
    
    print("\n📌 IMPORTANTE: Conforme o enunciado:")
    print("   - si1 (pressão sistólica): NÃO USAR (usada apenas no cálculo de si3)")
    print("   - si2 (pressão diastólica): NÃO USAR (usada apenas no cálculo de si3)")
    print("   - si3 (qPA): USAR")
    print("   - si4 (pulso): USAR")
    print("   - si5 (respiração): USAR")
    print("   - gi (gravidade): USAR")
    
    print(f"\nFeatures selecionadas para modelagem: {features_usar}")
    
    # Estatísticas descritivas das features selecionadas
    print("\nEstatísticas descritivas das features selecionadas:")
    print(df_train[features_usar].describe())
    
    # Correlação entre features
    print("\nMatriz de correlação entre features:")
    corr_matrix = df_train[features_usar].corr()
    print(corr_matrix)
    
    # Visualização da correlação
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Matriz de Correlação entre Features', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('correlacao_features.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Heatmap de correlação salvo em 'correlacao_features.png'")


In [ ]:
# Distribuições das features por classe
if df_train is not None:
    print("\n" + "="*80)
    print("DISTRIBUIÇÕES DAS FEATURES POR CLASSE DE GRAVIDADE")
    print("="*80)
    
    # Criar visualizações
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.ravel()
    
    for idx, feature in enumerate(features_usar):
        for classe in sorted(df_train['yi'].unique()):
            dados_classe = df_train[df_train['yi'] == classe][feature]
            axes[idx].hist(dados_classe, alpha=0.6, label=f'Classe {int(classe)}', bins=30)
        
        axes[idx].set_title(f'Distribuição de {feature} por Classe', fontweight='bold')
        axes[idx].set_xlabel(feature)
        axes[idx].set_ylabel('Frequência')
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)
    
    plt.suptitle('Distribuições das Features por Classe de Gravidade', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig('distribuicoes_por_classe.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Gráficos de distribuição salvos em 'distribuicoes_por_classe.png'")
    
    # Estatísticas por classe
    print("\nEstatísticas das features por classe:")
    for classe in sorted(df_train['yi'].unique()):
        print(f"\n{'='*60}")
        print(f"CLASSE {int(classe)}: {['Crítico', 'Instável', 'Potencialmente Estável', 'Estável'][int(classe)-1]}")
        print(f"{'='*60}")
        dados_classe = df_train[df_train['yi'] == classe][features_usar]
        print(dados_classe.describe())


## 3. Preparação dos Dados


In [ ]:
# Preparação dos dados
if df_train is not None:
    print("=" * 80)
    print("PREPARAÇÃO DOS DADOS")
    print("=" * 80)
    
    # Selecionar apenas as features que DEVEM ser usadas
    X = df_train[features_usar].copy()
    y = df_train['yi'].astype(int).copy()
    
    print(f"\nFeatures selecionadas: {features_usar}")
    print(f"Shape de X: {X.shape}")
    print(f"Shape de y: {y.shape}")
    print(f"\nDistribuição das classes:")
    print(y.value_counts().sort_index())
    
    # Verificar balanceamento
    balance_ratio = y.value_counts().min() / y.value_counts().max()
    print(f"\nRazão de balanceamento: {balance_ratio:.3f}")
    if balance_ratio < 0.5:
        print("⚠️ Dataset desbalanceado detectado")
    else:
        print("✅ Dataset relativamente balanceado")
    
    # Separação treino/validação (80/20)
    print("\n" + "="*80)
    print("SEPARAÇÃO TREINO/VALIDAÇÃO")
    print("="*80)
    print("Estratégia: 80% treino, 20% validação (stratified)")
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\n✅ Separação realizada:")
    print(f"  Treino: {X_train.shape[0]} amostras ({X_train.shape[0]/len(X)*100:.1f}%)")
    print(f"  Validação: {X_val.shape[0]} amostras ({X_val.shape[0]/len(X)*100:.1f}%)")
    
    print(f"\n  Distribuição de classes no TREINO:")
    print(y_train.value_counts().sort_index())
    print(f"\n  Distribuição de classes na VALIDAÇÃO:")
    print(y_val.value_counts().sort_index())
    
    # Normalização (importante para MLP)
    print("\n" + "="*80)
    print("NORMALIZAÇÃO DOS DADOS")
    print("="*80)
    print("Estratégia: StandardScaler (importante para MLP)")
    print("Justificativa: MLP é sensível à escala das features")
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    # Converter de volta para DataFrame
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=features_usar, index=X_train.index)
    X_val_scaled = pd.DataFrame(X_val_scaled, columns=features_usar, index=X_val.index)
    
    print(f"\n✅ Normalização realizada")
    print(f"  Médias após normalização (treino): {X_train_scaled.mean().round(3).to_dict()}")
    print(f"  Desvios padrão após normalização (treino): {X_train_scaled.std().round(3).to_dict()}")
    
    # Para Random Forest, não precisa normalização, mas não prejudica
    # Vamos usar dados normalizados para ambos para facilitar comparação
    X_train_final = X_train_scaled
    X_val_final = X_val_scaled


## 4. Ajuste dos Modelos de Classificação

### 4.1 Random Forest


In [ ]:
# MODELO 1: Random Forest
if df_train is not None:
    print("=" * 80)
    print("MODELO 1: RANDOM FOREST (FLORESTAS ALEATÓRIAS)")
    print("=" * 80)
    
    # Configurar Random Forest
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    
    print("\nParâmetros do modelo:")
    print(f"  - n_estimators: {rf.n_estimators}")
    print(f"  - max_depth: {rf.max_depth}")
    print(f"  - random_state: {rf.random_state}")
    
    # Validação cruzada
    print("\n" + "-"*80)
    print("VALIDAÇÃO CRUZADA (5-fold estratificado)")
    print("-"*80)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores_rf = cross_val_score(rf, X_train_final, y_train, cv=cv, scoring='accuracy')
    print(f"Accuracy média (CV): {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std() * 2:.4f})")
    
    # Treinar no conjunto completo de treino
    print("\n" + "-"*80)
    print("TREINAMENTO DO MODELO")
    print("-"*80)
    rf.fit(X_train_final, y_train)
    print("✅ Modelo treinado com sucesso!")
    
    # Predições no conjunto de validação
    print("\n" + "-"*80)
    print("PREDIÇÕES NO CONJUNTO DE VALIDAÇÃO")
    print("-"*80)
    y_pred_rf = rf.predict(X_val_final)
    y_pred_proba_rf = rf.predict_proba(X_val_final)
    
    # Métricas
    acc_rf = accuracy_score(y_val, y_pred_rf)
    prec_rf = precision_score(y_val, y_pred_rf, average='weighted', zero_division=0)
    rec_rf = recall_score(y_val, y_pred_rf, average='weighted', zero_division=0)
    f1_rf = f1_score(y_val, y_pred_rf, average='weighted', zero_division=0)
    
    print(f"\n📊 MÉTRICAS DO RANDOM FOREST:")
    print(f"  Accuracy:  {acc_rf:.4f}")
    print(f"  Precision: {prec_rf:.4f}")
    print(f"  Recall:    {rec_rf:.4f}")
    print(f"  F1-Score:  {f1_rf:.4f}")
    
    # Matriz de confusão
    cm_rf = confusion_matrix(y_val, y_pred_rf)
    print(f"\n📋 MATRIZ DE CONFUSÃO:")
    print(cm_rf)
    
    # Feature importance
    print(f"\n🔍 FEATURE IMPORTANCE:")
    feature_importance_rf = pd.DataFrame({
        'Feature': features_usar,
        'Importance': rf.feature_importances_
    }).sort_values('Importance', ascending=False)
    print(feature_importance_rf.to_string(index=False))
    
    # Salvar resultados
    resultados_rf = {
        'model': rf,
        'accuracy': acc_rf,
        'precision': prec_rf,
        'recall': rec_rf,
        'f1': f1_rf,
        'cv_scores': cv_scores_rf,
        'y_pred': y_pred_rf,
        'y_pred_proba': y_pred_proba_rf,
        'confusion_matrix': cm_rf,
        'feature_importance': feature_importance_rf
    }


In [ ]:
# MODELO 2: MLP (Multi-Layer Perceptron)
if df_train is not None:
    print("=" * 80)
    print("MODELO 2: MLP (MULTI-LAYER PERCEPTRON - REDE NEURAL)")
    print("=" * 80)
    
    # Configurar MLP
    mlp = MLPClassifier(
        hidden_layer_sizes=(100, 50),  # 2 camadas ocultas: 100 e 50 neurônios
        activation='relu',
        solver='adam',
        alpha=0.0001,  # Regularização L2
        batch_size='auto',
        learning_rate='constant',
        learning_rate_init=0.001,
        max_iter=500,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=10
    )
    
    print("\nParâmetros do modelo:")
    print(f"  - hidden_layer_sizes: {mlp.hidden_layer_sizes}")
    print(f"  - activation: {mlp.activation}")
    print(f"  - solver: {mlp.solver}")
    print(f"  - max_iter: {mlp.max_iter}")
    print(f"  - early_stopping: {mlp.early_stopping}")
    
    # Validação cruzada
    print("\n" + "-"*80)
    print("VALIDAÇÃO CRUZADA (5-fold estratificado)")
    print("-"*80)
    cv_scores_mlp = cross_val_score(mlp, X_train_final, y_train, cv=cv, scoring='accuracy')
    print(f"Accuracy média (CV): {cv_scores_mlp.mean():.4f} (+/- {cv_scores_mlp.std() * 2:.4f})")
    
    # Treinar no conjunto completo de treino
    print("\n" + "-"*80)
    print("TREINAMENTO DO MODELO")
    print("-"*80)
    mlp.fit(X_train_final, y_train)
    print("✅ Modelo treinado com sucesso!")
    print(f"  Iterações realizadas: {mlp.n_iter_}")
    print(f"  Loss final: {mlp.loss_:.4f}")
    
    # Predições no conjunto de validação
    print("\n" + "-"*80)
    print("PREDIÇÕES NO CONJUNTO DE VALIDAÇÃO")
    print("-"*80)
    y_pred_mlp = mlp.predict(X_val_final)
    y_pred_proba_mlp = mlp.predict_proba(X_val_final)
    
    # Métricas
    acc_mlp = accuracy_score(y_val, y_pred_mlp)
    prec_mlp = precision_score(y_val, y_pred_mlp, average='weighted', zero_division=0)
    rec_mlp = recall_score(y_val, y_pred_mlp, average='weighted', zero_division=0)
    f1_mlp = f1_score(y_val, y_pred_mlp, average='weighted', zero_division=0)
    
    print(f"\n📊 MÉTRICAS DO MLP:")
    print(f"  Accuracy:  {acc_mlp:.4f}")
    print(f"  Precision: {prec_mlp:.4f}")
    print(f"  Recall:    {rec_mlp:.4f}")
    print(f"  F1-Score:  {f1_mlp:.4f}")
    
    # Matriz de confusão
    cm_mlp = confusion_matrix(y_val, y_pred_mlp)
    print(f"\n📋 MATRIZ DE CONFUSÃO:")
    print(cm_mlp)
    
    # Salvar resultados
    resultados_mlp = {
        'model': mlp,
        'accuracy': acc_mlp,
        'precision': prec_mlp,
        'recall': rec_mlp,
        'f1': f1_mlp,
        'cv_scores': cv_scores_mlp,
        'y_pred': y_pred_mlp,
        'y_pred_proba': y_pred_proba_mlp,
        'confusion_matrix': cm_mlp
    }


## 5. Comparação dos Modelos


In [ ]:
# Comparação dos Modelos
if df_train is not None:
    print("=" * 80)
    print("COMPARAÇÃO DOS MODELOS")
    print("=" * 80)
    
    # Tabela comparativa
    comparacao = pd.DataFrame({
        'Modelo': ['Random Forest', 'MLP'],
        'Accuracy': [resultados_rf['accuracy'], resultados_mlp['accuracy']],
        'Precision': [resultados_rf['precision'], resultados_mlp['precision']],
        'Recall': [resultados_rf['recall'], resultados_mlp['recall']],
        'F1-Score': [resultados_rf['f1'], resultados_mlp['f1']],
        'CV Accuracy (média)': [resultados_rf['cv_scores'].mean(), resultados_mlp['cv_scores'].mean()],
        'CV Accuracy (std)': [resultados_rf['cv_scores'].std(), resultados_mlp['cv_scores'].std()]
    })
    
    print("\n📊 TABELA COMPARATIVA:")
    print(comparacao.to_string(index=False))
    
    # Identificar melhor modelo
    melhor_modelo = comparacao.loc[comparacao['F1-Score'].idxmax(), 'Modelo']
    print(f"\n🏆 MELHOR MODELO (por F1-Score): {melhor_modelo}")
    
    # Visualização: Matrizes de confusão lado a lado
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Random Forest
    sns.heatmap(resultados_rf['confusion_matrix'], annot=True, fmt='d', cmap='Blues', 
                ax=axes[0], xticklabels=sorted(y_val.unique()), yticklabels=sorted(y_val.unique()))
    axes[0].set_title(f'Random Forest\nAccuracy: {resultados_rf["accuracy"]:.3f}', fontweight='bold')
    axes[0].set_xlabel('Predito')
    axes[0].set_ylabel('Real')
    
    # MLP
    sns.heatmap(resultados_mlp['confusion_matrix'], annot=True, fmt='d', cmap='Oranges', 
                ax=axes[1], xticklabels=sorted(y_val.unique()), yticklabels=sorted(y_val.unique()))
    axes[1].set_title(f'MLP\nAccuracy: {resultados_mlp["accuracy"]:.3f}', fontweight='bold')
    axes[1].set_xlabel('Predito')
    axes[1].set_ylabel('Real')
    
    plt.suptitle('Matrizes de Confusão - Comparação dos Modelos', 
                 fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('matrizes_confusao_comparacao.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Matrizes de confusão salvas em 'matrizes_confusao_comparacao.png'")


In [ ]:
# Relatórios de classificação detalhados
if df_train is not None:
    print("\n" + "="*80)
    print("RELATÓRIOS DE CLASSIFICAÇÃO DETALHADOS")
    print("="*80)
    
    class_names = ['Crítico (1)', 'Instável (2)', 'Pot. Estável (3)', 'Estável (4)']
    
    print("\n" + "="*80)
    print("RANDOM FOREST")
    print("="*80)
    print(classification_report(y_val, resultados_rf['y_pred'], 
                                target_names=class_names))
    
    print("\n" + "="*80)
    print("MLP (REDE NEURAL)")
    print("="*80)
    print(classification_report(y_val, resultados_mlp['y_pred'], 
                                target_names=class_names))
    
    # Visualização comparativa de métricas
    fig, ax = plt.subplots(figsize=(12, 6))
    
    modelos = ['Random Forest', 'MLP']
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    rf_values = [resultados_rf['accuracy'], resultados_rf['precision'], 
                 resultados_rf['recall'], resultados_rf['f1']]
    mlp_values = [resultados_mlp['accuracy'], resultados_mlp['precision'], 
                  resultados_mlp['recall'], resultados_mlp['f1']]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, rf_values, width, label='Random Forest', color='steelblue', alpha=0.8)
    bars2 = ax.bar(x + width/2, mlp_values, width, label='MLP', color='orange', alpha=0.8)
    
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title('Comparação de Métricas: Random Forest vs MLP', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim([0, 1.1])
    
    # Adicionar valores nas barras
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('comparacao_metricas.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Gráfico comparativo de métricas salvo em 'comparacao_metricas.png'")


## 6. Aplicação no Teste Cego


In [ ]:
# Carregar e processar dados de teste cego
print("=" * 80)
print("APLICAÇÃO NO TESTE CEGO")
print("=" * 80)

file_test = 'teste_cego_com_classe.csv'

try:
    # Carregar arquivo de teste (sem labels)
    # Formato: i,si3,si4,si5,gi (separado por vírgulas, sem si1, si2, yi)
    df_test = pd.read_csv(file_test, sep=',', header=None, 
                         names=['i', 'si3', 'si4', 'si5', 'gi'])
    
    print(f"\n✅ Arquivo de teste carregado com sucesso!")
    print(f"Shape: {df_test.shape}")
    print(f"Linhas: {df_test.shape[0]}, Colunas: {df_test.shape[1]}\n")
    
    print("Primeiras linhas do dataset de teste:")
    print(df_test.head(10))
    
    # Preparar dados de teste (usar as mesmas features)
    X_test = df_test[features_usar].copy()
    
    print(f"\nFeatures selecionadas para teste: {features_usar}")
    print(f"Shape de X_test: {X_test.shape}")
    
    # Normalizar usando o mesmo scaler do treino
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=features_usar, index=X_test.index)
    
    print("\n✅ Dados de teste normalizados usando o scaler do treino")
    
    # Fazer predições com ambos os modelos
    print("\n" + "="*80)
    print("PREDIÇÕES NO TESTE CEGO")
    print("="*80)
    
    # Random Forest
    y_pred_test_rf = rf.predict(X_test_scaled)
    print(f"\n📊 Random Forest:")
    print(f"  Total de predições: {len(y_pred_test_rf)}")
    print(f"  Distribuição das predições:")
    print(pd.Series(y_pred_test_rf).value_counts().sort_index())
    
    # MLP
    y_pred_test_mlp = mlp.predict(X_test_scaled)
    print(f"\n📊 MLP:")
    print(f"  Total de predições: {len(y_pred_test_mlp)}")
    print(f"  Distribuição das predições:")
    print(pd.Series(y_pred_test_mlp).value_counts().sort_index())
    
    # Comparar predições dos dois modelos
    print(f"\n📊 COMPARAÇÃO DAS PREDIÇÕES:")
    concordancia = (y_pred_test_rf == y_pred_test_mlp).sum()
    discordancia = (y_pred_test_rf != y_pred_test_mlp).sum()
    print(f"  Predições concordantes: {concordancia} ({concordancia/len(y_pred_test_rf)*100:.1f}%)")
    print(f"  Predições discordantes: {discordancia} ({discordancia/len(y_pred_test_rf)*100:.1f}%)")
    
    # Salvar predições
    resultados_teste = pd.DataFrame({
        'i': df_test['i'],
        'si3': df_test['si3'],
        'si4': df_test['si4'],
        'si5': df_test['si5'],
        'gi': df_test['gi'],
        'predicao_RF': y_pred_test_rf,
        'predicao_MLP': y_pred_test_mlp
    })
    
    # Salvar em arquivo
    resultados_teste.to_csv('predicoes_teste_cego.csv', index=False)
    print(f"\n✅ Predições salvas em 'predicoes_teste_cego.csv'")
    
    # Mostrar primeiras predições
    print("\nPrimeiras 20 predições:")
    print(resultados_teste.head(20).to_string(index=False))
    
except FileNotFoundError:
    print(f"⚠️ Arquivo '{file_test}' não encontrado!")
    print("O arquivo de teste será processado quando estiver disponível.")
    df_test = None
except Exception as e:
    print(f"⚠️ Erro ao processar arquivo de teste: {e}")
    df_test = None


## 7. Resumo e Conclusões


In [ ]:
# Resumo Final e Conclusões
if df_train is not None:
    print("=" * 80)
    print("RESUMO FINAL E CONCLUSÕES")
    print("=" * 80)
    
    resumo = f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RESUMO DA ANÁLISE DE CLASSIFICAÇÃO DE SINAI VITAIS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. OBJETIVO:
   Classificar vítimas de acidentes em 4 classes de gravidade baseado em sinais vitais:
   - Classe 1: Crítico
   - Classe 2: Instável
   - Classe 3: Potencialmente Estável
   - Classe 4: Estável

2. FEATURES UTILIZADAS:
   - si3: Qualidade da pressão arterial (qPA) [-10, 10]
   - si4: Pulso [0, 200] bpm
   - si5: Frequência respiratória [0, 22] FpM
   - gi: Gravidade (valor calculado)

3. MODELOS AJUSTADOS:
   
   a) RANDOM FOREST:
      - Accuracy:  {resultados_rf['accuracy']:.4f}
      - Precision: {resultados_rf['precision']:.4f}
      - Recall:    {resultados_rf['recall']:.4f}
      - F1-Score:  {resultados_rf['f1']:.4f}
      - CV Accuracy: {resultados_rf['cv_scores'].mean():.4f} ± {resultados_rf['cv_scores'].std():.4f}
   
   b) MLP (REDE NEURAL):
      - Accuracy:  {resultados_mlp['accuracy']:.4f}
      - Precision: {resultados_mlp['precision']:.4f}
      - Recall:    {resultados_mlp['recall']:.4f}
      - F1-Score:  {resultados_mlp['f1']:.4f}
      - CV Accuracy: {resultados_mlp['cv_scores'].mean():.4f} ± {resultados_mlp['cv_scores'].std():.4f}

4. COMPARAÇÃO:
   - Melhor modelo (F1-Score): {melhor_modelo}
   - Diferença em F1-Score: {abs(resultados_rf['f1'] - resultados_mlp['f1']):.4f}
   - Ambos os modelos apresentaram performance {'similar' if abs(resultados_rf['f1'] - resultados_mlp['f1']) < 0.05 else 'diferente'}

5. FEATURE IMPORTANCE (Random Forest):
"""
    
    for idx, row in resultados_rf['feature_importance'].iterrows():
        resumo += f"   - {row['Feature']}: {row['Importance']:.4f}\n"
    
    resumo += f"""
6. CONCLUSÕES:
   - {'Random Forest' if melhor_modelo == 'Random Forest' else 'MLP'} apresentou melhor performance geral
   - As métricas indicam que o modelo consegue classificar adequadamente as vítimas
   - A validação cruzada mostra consistência nas predições
   - O modelo pode ser utilizado para classificar novas vítimas baseado nos sinais vitais

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""
    
    print(resumo)
